In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# data
leases = pd.read_csv('Leases.csv')
prices = pd.read_csv('Price and Availability Data.csv')

In [ ]:
# edit columns
market_mapping = {
    'Downtown Chicago': 'Chicago',
    'Denver-Boulder': 'Denver',
    'Suburban Maryland': 'Maryland',
    'Raleigh-Durham': 'Raleigh/Durham',
    'Dallas-Ft. Worth': 'Dallas/Ft Worth',
    'Orange County (CA)': 'Orange County',
    'South Bay': 'South Bay/San Jose',
    'Washington DC': 'Washington D.C.'
}
prices['market'] = prices['market'].replace(market_mapping)

leases['date'] = leases['year'].astype(str) + '_' + leases['quarter']
prices['date'] = prices['year'].astype(str) + '_' + prices['quarter']

In [ ]:
REGION = 'Midwest/Central'
MARKET = 'Denver'

rent_forecasts = {}
markets = ['Atlanta', 'Austin', 'Baltimore', 'Boston', 'Charlotte', 'Chicago', 'Chicago Suburbs', 'Dallas/Ft Worth', 'Denver', 'Detroit', 'Houston', 'Los Angeles', 'Manhattan', 'Nashville', 'Northern New Jersey', 'Northern Virginia', 'Orange County', 'Philadelphia', 'Phoenix', 'Raleigh/Durham', 'Salt Lake City', 'San Diego', 'San Francisco', 'Seattle', 'South Bay/San Jose', 'South Florida', 'Maryland', 'Tampa', 'Washington D.C.']

for market in markets:
    answer_A, answer_O = run_price_analysis(REGION, market)
    rent_forecasts[market] = (answer_A, answer_O)

In [ ]:
rent_forecasts

In [ ]:
def run_price_analysis(REGION, MARKET):

    print(f"Collect data for the {MARKET} market in the {REGION} region")
    region_data, market_data, market_data_from_price_dataset = break_up_data(REGION, MARKET)
    dataset = market_data_from_price_dataset
    
    dataset, removed = drop_na_rows('internal_class_rent', dataset)
    print(f"Dropped {removed} empty rows")
    
    print(f'Split by quality')
    A_dataset, O_dataset = split_by_quality(dataset)
    
    print('Plot prices')
    plot_prices(A_dataset, 'A', MARKET)
    plot_prices(O_dataset, 'O', MARKET)
    
    print("Forecast 2025 prices")
    A_prices = A_dataset['internal_class_rent'].values.tolist()
    O_prices = O_dataset['internal_class_rent'].values.tolist()
    
    A_futures = get_future_rent(A_prices)
    O_futures = get_future_rent(O_prices)
    
    return A_futures, O_futures

In [ ]:
def get_future_rent(rents):
    growth_rates = []
    
    # Calculate growth rates for the last 3 quarters
    for i in range(len(rents) - 4, len(rents) - 1):
        rate = (rents[i + 1] - rents[i]) / rents[i]
        growth_rates.append(rate)

    # Calculate average growth rate
    avg_growth_rate = sum(growth_rates) / len(growth_rates)

    # Generate predictions for 2025 Q1-Q4 using only growth rate method
    last_price = rents[-1]  # Start with 2024-Q4 price
    predictions_2025 = []

    for _ in range(4):
        next_price = last_price * (1 + avg_growth_rate)
        predictions_2025.append(round(next_price, 1))
        last_price = next_price  # Use unrounded value for next calculation

    return predictions_2025

In [ ]:
def break_up_data(REGION, MARKET):
    region   = leases[leases['region'] == REGION]
    market   = region[region['market'] == MARKET]
    market_p = prices[prices['market'] == MARKET]
    
    return (region, market, market_p)

In [ ]:
def split_by_quality(dataset):
    dataset_A = dataset[dataset['internal_class'] == 'A']
    dataset_O = dataset[dataset['internal_class'] == 'O']
    
    return (dataset_A, dataset_O)

In [ ]:
def drop_na_rows(column_name, dataset):
    original_count = len(dataset)
    dataset        = dataset.dropna(subset=[column_name])
    new_count =  len(dataset)

    removed_count = original_count - new_count
    return (dataset, removed_count)

In [ ]:
def plot_prices(dataset, quality, market):
    plt.figure(figsize=(10, 6))
    plt.plot(dataset['date'], dataset['internal_class_rent'], marker='o')
    plt.xlabel('Quarter')
    plt.ylabel('Rent')
    plt.title(f'Price over Time for {quality}-leases in {market}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()